In [ ]:
# Standard libraries of Python
import re

# Dependencies
import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup, Tag, NavigableString
from dataclasses import dataclass
from selectolax.parser import HTMLParser

# Libraries
from backend.database.parsers import parse_movements
from backend.database.src.creature import Pokemon, Mega_Pokemon
from backend.database.utils import functions

In [ ]:
# Class for selectolax scrap
# @dataclass
# class Pokemon:
#     name: str
#     gender: str
#     types: list

In [ ]:
#Selecto scrap
pokedex = []
url = 'https://www.serebii.net/pokedex-sm/{}.shtml'.format(str(1).zfill(3))
html = requests.get(url).text
tree = HTMLParser(html)
pokemon = Pokemon(name=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(1)').text(),
            gender=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(4)').text(),
            types=list(map(lambda x: x.attributes['alt'], tree.css('#content > main > div > div > table:nth-child(5) > tbody > tr:nth-child(2) > td.cen >> img'))))
pokedex.append(pokemon)

pd.DataFrame(pokedex)

In [ ]:
#Examble of use with bs4
html_text = requests.get(f'https://www.serebii.net/pokedex-xy/006.shtml').text
soup = BeautifulSoup(html_text, 'html.parser')
all_divs = soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[0].find_all('td', {'class': 'footype'})

In [ ]:
# Class test
x = Pokemon(8,'raichu')

all_divs = x.soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[1].find_all('table', {'class': 'dextable'})

x.name()

In [ ]:
# Egg moves in case of regional form pokémon, for the 8th generation and 7th generation, including BDSP data
# Class test
x = Pokemon(8,'raichu')

all_divs = x.soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[1].find_all('table', {'class': 'dextable'})

x.name()

info = []
for i in foo_info[15].find_all('td'):
    info.append(i)

data = info[1:]
org = [item for sublist in data for item in sublist]
reference = []
i = 7
while i < len(org):
    html = str(org[i])
    soup = BeautifulSoup(html, 'html.parser')

    img_tag = soup.find_all('img')
    if img_tag:
        values = [img.get('alt') for img in img_tag]
        del org[i]

        for pos,value in enumerate(values):
            if pos < 1:
                org.insert(i,value)
            else:
                org.insert(i+1,value)

        i += 10

    elif org[i].text == 'Details':
        del org[i]

        if not reference:
            reference.append(i-7)
        
        i += 8

    elif org[i+1].text == '':
        if x.p_name == 'Raichu' and org[i].text == 'Volt Tackle':
            del org[i+7]

            exclusive_move_case = org[i:]
            reference.append(i)

            i += 9

            continue
        
        if len(reference) == 1:
            reference.append(i)

        del org[i+1]
        del org[i+8]

        i += 9
    
    else:
        i -= 7

form_egg_moves = org[0:reference[0]]
eightgen_egg_moves = org[reference[0]:reference[1]]
bdsp_egg_moves = org[reference[1]:reference[2]]

form_egg_moves = pd.DataFrame([org[i:i+10] for i in range(0,reference[0],10)])
eightgen_egg_moves = pd.DataFrame([org[i:i+8] for i in range(reference[0],reference[1],8)])
bdsp_egg_moves = pd.DataFrame([org[i:i+9] for i in range(reference[1],reference[2],9)])

if exclusive_move_case:
    exclusive_move_case = pd.DataFrame([exclusive_move_case])
    eightgen_egg_moves = pd.concat([eightgen_egg_moves,exclusive_move_case], ignore_index=True)
bdsp_egg_moves[9] = bdsp_egg_moves[1]
bdsp_egg_moves = bdsp_egg_moves.drop(columns=1)
bdsp_egg_moves.columns = range(len(bdsp_egg_moves.columns))
eightgen_egg_moves = pd.concat([eightgen_egg_moves,bdsp_egg_moves], axis=0, ignore_index=True)

In [ ]:
# TM|TR|Technical Machine|Technical Record case for regional form pokémon in the 8th generation and 7th generation
# Class test
x = Pokemon(8,'raichu')

all_divs = x.soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[1].find_all('table', {'class': 'dextable'})

x.name()
x.elements()

info = []
for i in foo_info[12].find_all('td'):
    info.append(i)

data = info[1:]
org = [item for sublist in data for item in sublist]
org = list(filter(lambda x: 'table' not in str(x[1]), enumerate(org)))
org = list(map(lambda x: x[1], org))

df = []
i = 0
while i < len(org):
    l = parse_movements.list_lenght(i,org,'TM',x.p_elements)()

    if l == 10 and any(word in org[i+8].get('alt','') for word in ['Alolan', 'Galarian', 'Hisuian', 'Paldean', 'Normal']):
        fix = org[i:i+l]
        fix.insert(8 if 'Form' in org[i+8].get('alt','') else 9, 'N/A')
        df.extend([fix])
    else:
        df.extend([org[i:i+l]])
    i += l

In [ ]:
pd.DataFrame(df)

In [ ]:
# Max Moves and Z Moves data scrap from Serebii.net, for pokemon with regional forms
# Class test
x = Pokemon(8,'raichu')

all_divs = x.soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[1].find_all('table', {'class': 'dextable'})

x.name()
x.elements()

scrap = parse_movements.list_composition(foo_info[17])
positions, group = parse_movements.obtain_positions(scrap)
main_table = parse_movements.define_table(group,positions,scrap)
df = parse_movements.process_table_recursive(0,main_table,'Max Move',x.p_elements)

In [ ]:
df

In [ ]:
x = Pokemon(8,'raichu')

all_divs = x.soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[1].find_all('table', {'class': 'dextable'})

In [ ]:
x.name()
x.elements()

In [ ]:
x.p_elements

In [ ]:
info = []
for i in foo_info[12].find_all('td'):
    info.append(i)

data = info[1:]
org = [item for sublist in data for item in sublist]

In [ ]:
org = list(filter(lambda x: all(keyword not in str(x[1]) for keyword in ['table', '<br/>']), enumerate(org)))
org = list(map(lambda x: x[1], org))

In [ ]:
for n,i in enumerate(org):
    print(n,i)

In [ ]:
m = Mega_Pokemon(x)

In [ ]:
m.name()

In [ ]:
m.m_name

In [ ]:
s = all_divs[0].find_all('table', {'class': 'dextable'})

In [ ]:
for i, table in enumerate(s):
    if table.find('td', {'class': 'fooevo', 'colspan': '6'}):
        print('existe')
        print(i)

In [ ]:
#Weakness
types = []

for td_tag in foo_info[0:18]:
    a_tag = td_tag.find('img')
    if a_tag:
        type_text = a_tag['alt']
        types.append(type_text)

s = 'Attacking Move Type: ','-type'
for string in s:
    types = list(map(lambda x: x.replace(string,''),types))

print(types)

filtered = [tag for tag in foo_info[18:36] if '*' in tag.get_text(strip=True)]
filtered_t = [i.text for i in filtered]
val = list(map(lambda x: int(x.replace('*','')),filtered_t))

In [ ]:
#Elemental Types
tp = foo_info[0]
types = []
for tag in tp:
    a_tag = tag.find('img')
    print(a_tag)
    type_text = a_tag['alt']
    print(type_text)
    types.append(type_text)

s = 'Attacking Move Type: ','-type'
for string in s:
    types = list(map(lambda x: x.replace(string,''),types))

In [ ]:
base_form = []
form = []

if foo_info[0].find(string='Normal'):
    main = foo_info[0].find_all('td')

    for index,tag in enumerate(main):
        lazy = tag.text.strip()

        if lazy == 'Normal':
            basic = main[index+1].find('img')
            type_text = basic['alt']
            base_form.append(type_text)
        elif lazy:
            basic = main[index+1].find_all('img')
            for e in basic:
                type_text = e['alt']
                form.append(type_text)

In [ ]:
#Get al tables from the main center table
tables = all_divs[0].find_all('table', {'class': 'dextable'})

#Search for the line that contains <td class="fooevo" colspan="6">
mega_evolution_position = -1
counter = 0
for i, table in enumerate(tables):
    if table.find('td', {'class': 'fooevo', 'colspan': '6'}):
        mega_evolution_position = i
        print(i)
        print(table)
        break #Simulates return
    else:
        print('Does not have Mega')

In [ ]:
#Mega Evolution Types
t = tables[19].find('td', class_='cen')

ty_val = []

for i in t:
    n = i.find('img')
    if n and not isinstance(n,Tag):
        continue
    n = n['src']
    for element in x.elemental_types:
        ty = element.lower()
        if ty in n:
            ty_val.append(element)

print(ty_val)

In [ ]:
#Mega Evolution Weakness
w = []

if ty_val == x._elements:
    pass
else:
    weak_location = tables[22].find('td', string=re.compile("Base Stats - Total.*")).find_next_siblings('td')
    filtered_types = list(filter(lambda x: '*' in x.text, weak_location.find_all('td', {'class': 'footype'})))
    filtered = [tag.text for tag in filtered_types if '*' in tag.get_text(strip=True)]

    val = list(map(lambda x: x.replace('*',''),filtered_t))

    for i in val:
        try:
            w.append(int(i))
        except ValueError:
            w.append(float(i))

    mega_weak = dict(zip(x.elemental_types,w))